In [1]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2 as cv
import numpy as np
import mediapipe as mp

In [2]:
mp_face_mesh = mp.solutions.face_mesh
LEFT_EYE =[362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
RIGHT_EYE =[33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]
LEFT_IRIS = [474, 475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

In [3]:
cap = cv.VideoCapture(0)

In [ ]:
with mp_face_mesh.FaceMesh(
    max_num_faces = 1, 
    refine_landmarks=True,
    min_detection_confidence =0.5,
    min_tracking_confidence=0.5
) as face_mesh:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv.flip(frame, 1) # converts camera image into mirror image
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        img_h, img_w = frame.shape[:2]
        results = face_mesh.process(rgb_frame)
        if results.multi_face_landmarks:
            #print(results.multi_face_landmarks[0].landmark)
            #[print(p.x, p.y)for p in results.multi_face_landmarks[0].landmark]
            mesh_points =np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])
            #print(mesh_points.shape)
            #cv.polylines(frame, [mesh_points[LEFT_IRIS]], True, (0,255,0), 1, cv.LINE_AA)
            #cv.polylines(frame, [mesh_points[RIGHT_IRIS]], True, (0,255,0), 1, cv.LINE_AA) #eye landmarks, The border around
                            # iris is square.
            (l_cx, l_cy), l_radius= cv.minEnclosingCircle(mesh_points[LEFT_IRIS])
            (r_cx, r_cy), r_radius= cv.minEnclosingCircle(mesh_points[RIGHT_IRIS]) # This will give circular borderline and will
                                                                # cover whole of iris.
            center_left = np.array([l_cx, l_cy], dtype = np.int32) #L_cx and l_cy, r_cx, r_cy will be converted from floating to
                                                            # integral points.
            center_right = np.array([r_cx, r_cy], dtype = np.int32)
            cv.circle(frame, center_left, int(l_radius), (0,0,255), 1, cv.LINE_AA)
            cv.circle(frame, center_right, int(r_radius), (0,0,255), 1, cv.LINE_AA)
            
                                 
        cv.imshow('img', frame)
        key = cv.waitKey(1)
        if key == ord('q'):
            break
cap.release()
cv.destroyAllWindows()